### 前言
用python爬虫豆瓣top250电影榜单，包括电影名称、制片国家/地区、导演、主演、电影类型、上映日期、片长、评分、评分人数等信息，接下来详细介绍爬虫的过程

豆瓣网未登陆的情况下，html返回418，418的意思是被网站的反爬程序识别并返回的，没有添加请求头等信息，再次请求添加header的User-Agent信息，由于用本人账号(User-Agent和Cookie)采集，需要避免频繁采集，否则会被封号

In [1]:
#导入库
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

#### 处理User-Agent和Cookie

In [3]:
# 定义函数，用来处理User-Agent和Cookie
def ua_ck():
    '''
    网站需要登录才能采集的情况，需要从Network--Doc里复制User-Agent和Cookie，Cookie要转化为字典
    '''

    user_agent = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}

    cookies = 'Cookie: ll="118202"; bid=3qZMqUnMF3Q; _vwo_uuid_v2=DF054191ED9D9938883FDD775C4890316|d3f7058206492c3214d806aad99fa2d4; ct=y; _ga=GA1.2.46508803.1595647901; gr_user_id=2e7d3867-fff4-4d2d-9809-cd3380da9906; douban-fav-remind=1; viewed="27174411_2038599_30147778"; _gid=GA1.2.1310884515.1596598393; push_doumail_num=0; push_noty_num=0; __utmv=30149280.17799; __utmc=30149280; __utmc=223695111; dbcl2="177996890:5mPJRlRRT0o"; ck=2uj-; __utma=30149280.275695738.1595647901.1596675636.1596694652.29; __utmz=30149280.1596694652.29.25.utmcsr=accounts.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/passport/login; __utmt=1; __utmb=30149280.4.10.1596694652; _pk_ses.100001.4cf6=*; _pk_ref.100001.4cf6=%5B%22%22%2C%22%22%2C1596694699%2C%22https%3A%2F%2Fwww.douban.com%2F%22%5D; __utma=223695111.46508803.1595647901.1596675636.1596694699.13; __utmb=223695111.0.10.1596694699; __utmz=223695111.1596694699.13.9.utmcsr=douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/; _pk_id.100001.4cf6=ae4b3783198936af.1595647902.13.1596694960.1596678264.'

    # Cookie转化为字典
    cookies = cookies.split('; ')
    cookies_dict = {}
    for i in cookies:
        cookies_dict[i.split('=')[0]] = i.split('=')[1]

    return user_agent, cookies_dict

In [4]:
# 定义函数，用于获取豆瓣top250每一个页面的链接
def get_urls(n):
    '''
    n:页面数量，总共有25个页面
    '''

    urls = []
    num = (n-1)*25+1
    for i in range(0, num, 25):
        url = 'https://movie.douban.com/top250?start={}&filter='.format(i)
        urls.append(url)

    return urls


# 总共有10个页面，每个页面25部电影
get_urls(10)

['https://movie.douban.com/top250?start=0&filter=',
 'https://movie.douban.com/top250?start=25&filter=',
 'https://movie.douban.com/top250?start=50&filter=',
 'https://movie.douban.com/top250?start=75&filter=',
 'https://movie.douban.com/top250?start=100&filter=',
 'https://movie.douban.com/top250?start=125&filter=',
 'https://movie.douban.com/top250?start=150&filter=',
 'https://movie.douban.com/top250?start=175&filter=',
 'https://movie.douban.com/top250?start=200&filter=',
 'https://movie.douban.com/top250?start=225&filter=']

In [5]:
# 定义函数，获取每个页面25部电影的链接
def get_movies_url(url, u_a, c_d):
    '''
    url：每一个页面的链接
    u_a：User-Agent
    c_d：cookies
    '''

    html = requests.get(url,
                        headers=u_a,  # 加载User-Agent
                        cookies=c_d)  # 加载cookie

    html.encoding = html.apparent_encoding  # 解决乱码的万金油方法

    if html.status_code == 200:
        print('网页访问成功，代码：{}\n'.format(html.status_code))

    soup = BeautifulSoup(html.text, 'html.parser')  # 用 html.parser 来解析网页
    items = soup.find('ol', class_='grid_view').find_all('li')
    movies_url = []

    for item in items:
        # 电影链接
        movie_href = item.find('div', class_='hd').find('a')['href']
        movies_url.append(movie_href)

    return movies_url
    time.sleep(0.4)    # 设置时间间隔，0.4秒采集一次，避免频繁登录网页

In [6]:
# 定义函数，获取每一部电影的详细信息
def get_movie_info(href, u_a, c_d):
    '''
    href：每一部电影的链接
    u_a：User-Agent
    c_d：cookies
    '''

    html = requests.get(href,
                        headers=u_a,
                        cookies=c_d)
    soup = BeautifulSoup(html.text, 'html.parser')  # 用 html.parser 来解析网页
    item = soup.find('div', id='content')

    movie = {}  # 新建字典，存放电影信息

    # 电影名称
    movie['电影名称'] = item.h1.span.text

    # 导演、类型、制片国家/地区、语言、上映时间、片长（部分电影这些信息不全，先全部采集，留待数据分析时处理）
    movie['电影其他信息'] = item.find(
        'div', id='info').text.replace(' ', '').split('\n')
    for i in movie['电影其他信息']:
        if ':' in i:
            movie[i.split(':')[0]] = i.split(':')[1]
        else:
            continue

    # 豆瓣评分、评分人数
    movie['评分'] = item.find('div', id='interest_sectl').find(
        'div', class_='rating_self clearfix').find('strong', class_='ll rating_num').text
    movie['评分人数'] = item.find('div', id='interest_sectl').find('div', class_='rating_self clearfix').find(
        'div', class_='rating_sum').find('span', property='v:votes').text

    return movie
    time.sleep(0.4)  # 0.4秒采集一次，避免频繁登录网页

In [7]:
# 设置主函数，运行上面设置好的函数
def main(n):
    '''
    n:页面数量，总共有10个页面
    u_a：User-Agent
    c_d：cookies
    '''
    print('开始采集数据，预计耗时2分钟')

    # 处理User-Agent和Cookie
    login = ua_ck()
    u_a = login[0]
    c_d = login[1]

    # 获取豆瓣top250每一页的链接，共10页
    urls = get_urls(n)
    print('豆瓣10个网页链接已生成！！')

    # 获取每一页25部电影的链接，共250部
    top250_urls = []
    for url in urls:
        result = get_movies_url(url, u_a, c_d)
        top250_urls.extend(result)
    print('250部电影链接采集完成！！开始采集每部电影的详细信息.......')

    # 获取每一部电影的详细信息
    top250_movie = []  # 储存每部电影的信息
    error_href = []  # 储存采集错误的网址

    for href in top250_urls:
        try:
            movie = get_movie_info(href, u_a, c_d)
            top250_movie.append(movie)
        except:
            error_href.append(href)
            print('采集失败，失败网址是{}'.format(href))

    print('电影详细信息采集完成！！总共采集{}条数据'.format(len(top250_movie)))
    return top250_movie, error_href

In [8]:
# 启动主函数，开始采集数据
result = main(10)
result

开始采集数据，预计耗时2分钟
豆瓣10个网页链接已生成！！
网页访问成功，代码：200

网页访问成功，代码：200

网页访问成功，代码：200

网页访问成功，代码：200

网页访问成功，代码：200

网页访问成功，代码：200

网页访问成功，代码：200

网页访问成功，代码：200

网页访问成功，代码：200

网页访问成功，代码：200

250部电影链接采集完成！！开始采集每部电影的详细信息
电影详细信息采集完成！！总共采集250条数据


([{'电影名称': '肖申克的救赎 The Shawshank Redemption',
   '电影其他信息': ['',
    '导演:弗兰克·德拉邦特',
    '编剧:弗兰克·德拉邦特/斯蒂芬·金',
    '主演:蒂姆·罗宾斯/摩根·弗里曼/鲍勃·冈顿/威廉姆·赛德勒/克兰西·布朗/吉尔·贝罗斯/马克·罗斯顿/詹姆斯·惠特摩/杰弗里·德曼/拉里·布兰登伯格/尼尔·吉恩托利/布赖恩·利比/大卫·普罗瓦尔/约瑟夫·劳格诺/祖德·塞克利拉/保罗·麦克兰尼/芮妮·布莱恩/阿方索·弗里曼/V·J·福斯特/弗兰克·梅德拉诺/马克·迈尔斯/尼尔·萨默斯/耐德·巴拉米/布赖恩·戴拉特/唐·麦克马纳斯',
    '类型:剧情/犯罪',
    '制片国家/地区:美国',
    '语言:英语',
    '上映日期:1994-09-10(多伦多电影节)/1994-10-14(美国)',
    '片长:142分钟',
    '又名:月黑高飞(港)/刺激1995(台)/地狱诺言/铁窗岁月/消香克的救赎',
    'IMDb链接:tt0111161',
    ''],
   '导演': '弗兰克·德拉邦特',
   '编剧': '弗兰克·德拉邦特/斯蒂芬·金',
   '主演': '蒂姆·罗宾斯/摩根·弗里曼/鲍勃·冈顿/威廉姆·赛德勒/克兰西·布朗/吉尔·贝罗斯/马克·罗斯顿/詹姆斯·惠特摩/杰弗里·德曼/拉里·布兰登伯格/尼尔·吉恩托利/布赖恩·利比/大卫·普罗瓦尔/约瑟夫·劳格诺/祖德·塞克利拉/保罗·麦克兰尼/芮妮·布莱恩/阿方索·弗里曼/V·J·福斯特/弗兰克·梅德拉诺/马克·迈尔斯/尼尔·萨默斯/耐德·巴拉米/布赖恩·戴拉特/唐·麦克马纳斯',
   '类型': '剧情/犯罪',
   '制片国家/地区': '美国',
   '语言': '英语',
   '上映日期': '1994-09-10(多伦多电影节)/1994-10-14(美国)',
   '片长': '142分钟',
   '又名': '月黑高飞(港)/刺激1995(台)/地狱诺言/铁窗岁月/消香克的救赎',
   'IMDb链接': 'tt0111161',
   '评分': '9.7',
   '评分人数': '2100927'},
  {'电影名称': '霸王别姬',
   '电影其

In [9]:
df = pd.DataFrame(result[0])
df

电影名称  \
0    肖申克的救赎 The Shawshank Redemption   
1                               霸王别姬   
2                  阿甘正传 Forrest Gump   
3                       这个杀手不太冷 Léon   
4                      泰坦尼克号 Titanic   
..                               ...   
245             黑鹰坠落 Black Hawk Down   
246                   网络谜踪 Searching   
247                             四个春天   
248   黑客帝国2：重装上阵 The Matrix Reloaded   
249           发条橙 A Clockwork Orange   

                                                电影其他信息               导演  \
0    [, 导演:弗兰克·德拉邦特, 编剧:弗兰克·德拉邦特/斯蒂芬·金, 主演:蒂姆·罗宾斯/摩...         弗兰克·德拉邦特   
1    [, 导演:陈凯歌, 编剧:芦苇/李碧华, 主演:张国荣/张丰毅/巩俐/葛优/英达/蒋雯丽/...              陈凯歌   
2    [, 导演:罗伯特·泽米吉斯, 编剧:艾瑞克·罗斯/温斯顿·格鲁姆, 主演:汤姆·汉克斯/罗...         罗伯特·泽米吉斯   
3    [, 导演:吕克·贝松, 编剧:吕克·贝松, 主演:让·雷诺/娜塔莉·波特曼/加里·奥德曼/...            吕克·贝松   
4    [, 导演:詹姆斯·卡梅隆, 编剧:詹姆斯·卡梅隆, 主演:莱昂纳多·迪卡普里奥/凯特·温丝...          詹姆斯·卡梅隆   
..                                                 ...              ...   
245  [, 导演:雷德利·斯科特, 编剧:肯·诺兰/马克·鲍登, 主演:乔什·哈奈特/伊万·麦克格...          雷德利·斯科特   
246  [, 导演:阿尼什·查甘蒂, 编剧:阿尼什·查甘蒂/赛弗·奥哈尼安, 主演:约翰·赵/米切尔...          阿尼什·查甘蒂   
247  [, 导演:陆庆屹, 主演:陆运坤/李桂贤/陆庆伟/陆庆松/陆庆屹, 类型:纪录片/家庭, ...              陆庆屹   
248  [, 导演:莉莉·沃卓斯基/拉娜·沃卓斯基, 编剧:莉莉·沃卓斯基/拉娜·沃卓斯基, 主演:...  莉莉·沃卓斯基/拉娜·沃卓斯基   
249  [, 导演:斯坦利·库布里克, 编剧:斯坦利·库布里克/安东尼·伯吉斯, 主演:马尔科姆·麦...         斯坦利·库布里克   

                   编剧                                                 主演  \
0      弗兰克·德拉邦特/斯蒂芬·金  蒂姆·罗宾斯/摩根·弗里曼/鲍勃·冈顿/威廉姆·赛德勒/克兰西·布朗/吉尔·贝罗斯/马克·罗...   
1              芦苇/李碧华  张国荣/张丰毅/巩俐/葛优/英达/蒋雯丽/吴大维/吕齐/雷汉/尹治/马明威/费振翔/智一桐/...   
2      艾瑞克·罗斯/温斯顿·格鲁姆  汤姆·汉克斯/罗宾·怀特/加里·西尼斯/麦凯尔泰·威廉逊/莎莉·菲尔德/海利·乔·奥斯蒙/迈...   
3               吕克·贝松  让·雷诺/娜塔莉·波特曼/加里·奥德曼/丹尼·爱罗/彼得·阿佩尔/迈克尔·巴达鲁科/艾伦·格...   
4             詹姆斯·卡梅隆  莱昂纳多·迪卡普里奥/凯特·温丝莱特/比利·赞恩/凯西·贝茨/弗兰西丝·费舍/格劳瑞亚·斯图...   
..                ...                                                ...   
245        肯·诺兰/马克·鲍登  乔什·哈奈特/伊万·麦克格雷格/汤姆·塞兹摩尔/金·寇兹/艾文·布莱纳/艾瑞克·巴纳/休·丹...   
246   阿尼什·查甘蒂/赛弗·奥哈尼安  约翰·赵/米切尔·拉/黛博拉·梅辛/约瑟夫·李/萨拉·米博·孙/亚历克丝·杰恩·高/刘玥辰/...   
247               NaN                                陆运坤/李桂贤/陆庆伟/陆庆松/陆庆屹   
248   莉莉·沃卓斯基/拉娜·沃卓斯基  基努·里维斯/劳伦斯·菲什伯恩/凯瑞-安·莫斯/雨果·维文/莫妮卡·贝鲁奇/赫尔穆特·巴凯蒂...   
249  斯坦利·库布里克/安东尼·伯吉斯            马尔科姆·麦克道威尔/帕特里克·马基/迈克尔·贝茨/沃伦·克拉克/约翰·克莱夫   

              类型    制片国家/地区             语言  \
0          剧情/犯罪         美国             英语   
1       剧情/爱情/同性  中国大陆/中国香港          汉语普通话   
2          剧情/爱情         美国             英语   
3       剧情/动作/犯罪      法国/美国     英语/意大利语/法语   
4       剧情/爱情/灾难         美国  英语/意大利语/德语/俄语   
..           ...        ...            ...   
245     动作/历史/战争         美国   英语/阿拉伯语/索马里语   
246  剧情/悬疑/惊悚/犯罪     美国/俄罗斯             英语   
247       纪录片/家庭       中国大陆          贵州独山话   
248        动作/科幻    美国/澳大利亚          英语/法语   
249     剧情/科幻/犯罪      英国/美国             英语   

                                                  上映日期  \
0                    1994-09-10(多伦多电影节)/1994-10-14(美国)   
1                    1993-01-01(中国香港)/1993-07-26(中国大陆)   
2                     1994-06-23(洛杉矶首映)/1994-07-06(美国)   
3                                       1994-09-14(法国)   
4    1998-04-03(中国大陆)/1997-11-01(东京电影节)/1997-12-19(美国)   
..                                                 ...   
245                                     2001-12-18(美国)   
246  2018-12-14(中国大陆)/2018-01-20(圣丹斯电影节)/2018-08-24...   
247  2019-01-04(中国大陆)/2017-12-30(UCCA艺术放映)/2018-07-...   
248  2003-07-18(中国大陆)/2003-05-07(韦斯特伍德首映)/2003-05-1...   
249                    1971-12-19(纽约首映)/1972-01-13(英国)   

                             片长  \
0                         142分钟   
1            171分钟/155分钟(美国剧场版)   
2                         142分钟   
3         110分钟(剧场版)/133分钟(国际版)   
4              194分钟/227分钟(白星版)   
..                          ...   
245  144分钟/142分钟(德国)/152分钟(加长版)   
246                       102分钟   
247                       105分钟   
248                       138分钟   
249                       136分钟   

                                                    又名     IMDb链接 

In [10]:
# 保存为本地Excel文件
df.to_excel('豆瓣top250电影.xlsx')